<a href="https://colab.research.google.com/github/engeljason/cloud_colab_processing/blob/main/colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.2.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://security.ubuntu.com/ubuntu bionic-security InRelease
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:13 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad.net/graphics-drivers/ppa/ubu

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-10-24 05:55:20--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.51MB/s    in 0.2s    

2021-10-24 05:55:21 (5.51 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [29]:
!wget https://raw.githubusercontent.com/engeljason/cloud_colab_processing/main/Resources/schema.sql

--2021-10-24 07:01:38--  https://raw.githubusercontent.com/engeljason/cloud_colab_processing/main/Resources/schema.sql
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 637 [text/plain]
Saving to: ‘schema.sql’

schema.sql          100%[===================>]     637  --.-KB/s    in 0s      

2021-10-24 07:01:38 (34.5 MB/s) - ‘schema.sql’ saved [637/637]



In [2]:
files = ['https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz', 'https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Software_v1_00.tsv.gz']
filenames = ['amazon_reviews_us_Video_Games_v1_00.tsv.gz', 'amazon_reviews_us_Software_v1_00.tsv.gz']
i = 0

In [4]:
!wget -q https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz
!gunzip amazon_reviews_us_Video_Games_v1_00.tsv.gz

gzip: amazon_reviews_us_Video_Games_v1_00.tsv already exists; do you wish to overwrite (y or n)? n
	not overwritten


In [3]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","postgresql-42.2.9.jar").getOrCreate()

In [4]:
with open('amazon_reviews_us_Video_Games_v1_00.tsv') as file:
  print(file.readlines(5))

['marketplace\tcustomer_id\treview_id\tproduct_id\tproduct_parent\tproduct_title\tproduct_category\tstar_rating\thelpful_votes\ttotal_votes\tvine\tverified_purchase\treview_headline\treview_body\treview_date\n']


In [5]:
from pyspark import SparkFiles
url = files[i]
spark.sparkContext.addFile(url)

In [6]:
df = spark.read.csv(SparkFiles.get(filenames[i]), sep="\t", header=True, inferSchema=True)

In [7]:
df.dtypes

[('marketplace', 'string'),
 ('customer_id', 'int'),
 ('review_id', 'string'),
 ('product_id', 'string'),
 ('product_parent', 'int'),
 ('product_title', 'string'),
 ('product_category', 'string'),
 ('star_rating', 'int'),
 ('helpful_votes', 'int'),
 ('total_votes', 'int'),
 ('vine', 'string'),
 ('verified_purchase', 'string'),
 ('review_headline', 'string'),
 ('review_body', 'string'),
 ('review_date', 'string')]

In [9]:
from pyspark.sql.types import DateType
df2 = df.withColumn('review_date' , df['review_date'].cast(DateType()))

In [10]:
df2.dtypes

[('marketplace', 'string'),
 ('customer_id', 'int'),
 ('review_id', 'string'),
 ('product_id', 'string'),
 ('product_parent', 'int'),
 ('product_title', 'string'),
 ('product_category', 'string'),
 ('star_rating', 'int'),
 ('helpful_votes', 'int'),
 ('total_votes', 'int'),
 ('vine', 'string'),
 ('verified_purchase', 'string'),
 ('review_headline', 'string'),
 ('review_body', 'string'),
 ('review_date', 'date')]

In [12]:
review_id_df = df2[['review_id', 'customer_id', 'product_id' , 'product_parent' , 'review_date']]
review_id_df.head(1)

[Row(review_id='RTIS3L2M1F5SM', customer_id=12039526, product_id='B001CXYMFS', product_parent=737716809, review_date=datetime.date(2015, 8, 31))]

In [39]:
products_df = df2[['product_id' , 'product_title']].dropDuplicates()

In [25]:
customers_df = df2.groupBy('customer_id').count().withColumnRenamed('count', 'customer_count')
customers_df.head()

Row(customer_id=48670265, customer_count=1)

In [27]:
vine_df = df2[['review_id' , 'star_rating' , 'helpful_votes' , 'total_votes' , 'vine']]

In [36]:
import psycopg2

try:
  conn = psycopg2.connect(database = 'postgres', user = 'postgres', password = 'postgres', host = 'jasonhwrds.crsbnxfeuino.us-west-1.rds.amazonaws.com', port = '5432')
except:
  print("unable to connect")

cur = conn.cursor()

with open('schema.sql') as file:
  qry = file.read()
  try:
    cur.execute(qry)
  except:
    print("Create table failed")

conn.commit()
conn.close()
cur.close()


CREATE TABLE review_id_table (
  review_id TEXT PRIMARY KEY NOT NULL,
  customer_id INTEGER,
  product_id TEXT,
  product_parent INTEGER,
  review_date DATE -- this should be in the formate yyyy-mm-dd
);

-- This table will contain only unique values
CREATE TABLE products (
  product_id TEXT PRIMARY KEY NOT NULL UNIQUE,
  product_title TEXT
);

-- Customer table for first data set
CREATE TABLE customers (
  customer_id INT PRIMARY KEY NOT NULL UNIQUE,
  customer_count INT
);

-- vine table
CREATE TABLE vine_table (
  review_id TEXT PRIMARY KEY,
  star_rating INTEGER,
  helpful_votes INTEGER,
  total_votes INTEGER,
  vine TEXT
);



In [31]:
mode="append"
jdbc_url = "jdbc:postgresql://jasonhwrds.crsbnxfeuino.us-west-1.rds.amazonaws.com:5432/postgres"
config = {"user":"postgres",
          "password": "postgres",
          "driver":"org.postgresql.Driver"}

In [37]:
# Write DataFrame to table
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [40]:
products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [41]:
customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [42]:
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)